In [1]:
import json
import tarfile
import urllib.request
import pyconll
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
NAME='norec'
VERSION='1.0.1'
ARCHIVE=f'{NAME}-{VERSION}.tar.gz'
URL=f'https://www.mn.uio.no/ifi/english/research/projects/sant/data/norec/{ARCHIVE}'
DATA_DIR=Path('../data/norec_raw')
CONTENT_DIR=DATA_DIR / 'norec'
SAMPLE_FILE = CONTENT_DIR / 'metadata.json'
CONLLU_DIR = DATA_DIR / 'conllu'
TEXT_DIR = DATA_DIR / 'text'
FINAL_DIR = DATA_DIR / '..' / 'norec'

First we need to download the NoReC archive from UIO and extract it.
There's also a couple of tarballs inside the archive tarball.
We extract the conllu tarball which contains the review texts.

See https://github.com/ltgoslo/norec for Terms of Use.

In [3]:
if not SAMPLE_FILE.exists():
    if not DATA_DIR.exists():
        DATA_DIR.mkdir(parents=True)
    urllib.request.urlretrieve(URL, DATA_DIR / ARCHIVE)
    tarball = tarfile.open(DATA_DIR / ARCHIVE)
    tarball.extractall(CONTENT_DIR)
    tarball = tarfile.open(CONTENT_DIR / 'conllu.tar.gz')
    tarball.extractall(DATA_DIR)

Now, we extract the text from the CONLLU-files and put store in pure text files

In [4]:
for subset in ['train', 'dev', 'test']:
    if not (TEXT_DIR / subset).exists():
        (TEXT_DIR / subset).mkdir(parents=True)
    for child in (CONLLU_DIR / subset).iterdir():
        conllu = pyconll.load_from_file(child)
        with Path(str(child).replace('conllu', 'text', 1).replace('conllu', 'txt')).open(mode='w') as output:
            output.write('. '.join([sentence.text for sentence in conllu]))

The sample has the metadata for the reviews.

In [5]:
with (SAMPLE_FILE).open(encoding='utf-8') as input_file:
    sample_list = json.load(input_file)

In [6]:
len(sample_list)

35189

In [7]:
sample_list['000000']

{'authors': ['Birger Vestmo'],
 'category': 'screen',
 'day': 27,
 'excerpt': 'Toppen innen tv-drama akkurat nå!',
 'id': 0,
 'language': 'nb',
 'month': 9,
 'rating': 6,
 'source': 'p3',
 'source-category': 'tv',
 'source-id': 74781,
 'source-tags': [],
 'split': 'train',
 'tags': ['tv'],
 'title': 'Rome S02',
 'url': 'http://p3.no/filmpolitiet/rome-02',
 'year': 2007}

In [8]:
sample_list['000000']['rating']

6

In [9]:
negative_samples = {key: value for key, value in sample_list.items() if value['rating'] < 3}

In [10]:
len(negative_samples)

2787

In [11]:
positive_samples = {key: value for key, value in sample_list.items() if value['rating'] > 4}

In [12]:
len(positive_samples)

14725

In [13]:
class SampleLoader():
    # We ignore average reviews.
    NEGATIVE_THRESHOLD = 3
    POSTIVE_THRESHOLD = 4

    def __init__(self, sample_path: Path):
        self.path = sample_path
        
    def load(self, sample_id: str):
        full_path = self.path / f'{sample_id}.txt'
        with full_path.open() as input_file:
            text = input_file.read()
        return text

    @classmethod
    def categorise(cls, rating):
        return 1 if cls.positive_rating(rating) else 0

    @classmethod
    def negative_rating(cls, rating):
        return rating < cls.NEGATIVE_THRESHOLD

    @classmethod
    def positive_rating(cls, rating):
        return rating > cls.POSTIVE_THRESHOLD

    @classmethod
    def certain_value(cls, rating):
        return cls.negative_rating(rating) or cls.positive_rating(rating)

    def make_dataframe(self, samples):
        texts = [self.load(key) for key in samples.keys()]
        ratings = [self.categorise(value['rating']) for value in samples.values()]
        df = pd.DataFrame({'text': texts, 'rating': ratings})
        return df.sample(frac=1.)

In [14]:
loader = SampleLoader(TEXT_DIR / 'train')

In [15]:
loader.load('000000')

'Rome S02. Den andre og siste sesongen av Rome er ute på DVD i Norge.. Om du så sesong 1, vet du at du har noe stort i vente.. Har du aldri sett Rome før, stikk ut og kjøp begge sesongene.. Dette er nemlig en av verdens beste tv-serier, og etter å ha sett de fire første episodene av sesong 2, konstaterer jeg at kvaliteten ser ut til å holde seg på et nesten overraskende høyt nivå!. Enda mørkere. Sesong 2 starter nøyaktig der sesong 1 sluttet.. Julius Cæsar ligger myrdet i Senatet og Lucius Vorenus hulker over liket av Neobie.. Så blir historien enda mørkere.. Marcus Antonius tar over styringen av Roma, men utfordres fra uventet hold.. Og Lucius Vorenus blir rammet av nok en tragedie som sender ham ut på et selvdestruktivt spor og setter vennskapet med Titus Pullo i fare.. Interessant utvikling. Etter førstesesongens klimaks var jeg usikker på hvordan historien skulle ta oss med videre, men det tar ikke lang tid før sesong 2s nøkkelfigurer posisjonerer seg.. Den første episoden tar for 

Filter out average reviews and create dataframes for each subset

In [16]:
subset_names = ['train', 'test', 'dev']
loaders = {name: SampleLoader(TEXT_DIR / name) for name in subset_names}
with Path(SAMPLE_FILE).open(encoding='utf-8') as input_file:
    sample_list = json.load(input_file)
certain_samples = {key: value for key, value in sample_list.items() if SampleLoader.certain_value(value['rating'])}
subsets = {name: {key: value for key, value in certain_samples.items() if value['split'] == name} for name in subset_names}
dataframes = {name: loaders[name].make_dataframe(subset) for name, subset in subsets.items()}

In [17]:
dataframes['train'].head(10)

,text,rating
2676,«Poison». Som alle store artister passer Timbe...,1
12603,Totalt bortkasta Liv Tyler. Men denne filmen e...,0
6845,"TEST:. Ford B-Max - liten, men stor. B-Max er ...",1
2433,Teateranmeldelse:. «Litt av et par!». «Litt av...,1
5987,"Episk kraftpakke. Pen grafikk, vakre miljøer, ...",1
7707,Plextor Premium. Plextor er på ingen måte førs...,1
5319,Filmanmeldelse:. Intenst rasedrama. 1960-talle...,1
5805,Øya-anmeldelse:. Skarpe Diem. KONSERT: Karpe D...,1
4439,Sterk Oscar-vinner. Skarpt og gripende fra den...,1
12240,"Gere på Golgata, god påske. «Bedrageren» Arbit...",1


In [18]:
len(dataframes)

3

In [19]:
dataframes.keys()

dict_keys(['train', 'test', 'dev'])

In [20]:
for name, subset in dataframes.items():
    subset.to_pickle(FINAL_DIR / f'norsk_kategori_{name}.pkl')